In [1]:
import requests
import json
import os

In [2]:
url = "https://create.demo.sas.com"

def _connect_to_instance(refresh_token, verification):
	auth_url = f"{url}/SASLogon/oauth/token"
	# reading long-lived refresh token from txt file

	payload=f'grant_type=refresh_token&refresh_token={refresh_token}'
	headers = {
	'Accept': 'application/json',
	'Content-Type': 'application/x-www-form-urlencoded',
	'Authorization': 'Basic c2FzLmNsaTo=',
	}

	response = requests.request("POST", auth_url, headers=headers, data=payload, verify=verification)
	access_token = response.json()['access_token']
	return access_token

def _generate_access_token(auth_code, verification): 
	server = f"{url}/SASLogon/oauth/token"

	payload = f'grant_type=authorization_code&code={auth_code}'
	headers = {
		'Accept': 'application/json',
		'Content-Type': 'application/x-www-form-urlencoded',
		'Authorization': 'Basic c2FzLmNsaTo='
	}

	response = requests.request("POST", server, headers=headers, data=payload, verify=verification)

	# Parse the response text
	response_json = json.loads(response.text)

	# Get the refresh token
	refresh_token = response_json['refresh_token']
	return refresh_token

def get_connection(verify=False):
	if 'refresh_token.txt' not in [i.name for i in os.scandir()]:
		with open('refresh_token.txt', 'w') as file:
			file.write('test')
	if verify:
		os.environ['CAS_CLIENT_SSL_CA_LIST'] = 'ssemonthly-rootCA-Intermidiates_4CLI.pem'
		verify = 'ssemonthly-rootCA-Intermidiates_4CLI.pem'
	try:
		with open('refresh_token.txt', 'r') as token:
			refresh_token = token.read()
		access_token = _connect_to_instance(refresh_token, verify)

	except:
		print('https://create.demo.sas.com/SASLogon/oauth/authorize?client_id=sas.cli&response_type=code')
		auth_code = input('Please provide your access token by going to https://create.demo.sas.com/SASLogon/oauth/authorize?client_id=sas.cli&response_type=code:')
		refresh_token = _generate_access_token(auth_code, verify)
		with open('refresh_token.txt', 'w') as file:
			file.write(refresh_token)
		access_token = _connect_to_instance(refresh_token, verify)
	print('Connected!')
	return access_token

In [3]:
access_token = get_connection(verify=True)

Connected!


In [4]:
from sasctl import Session, pzmm
from sasctl.services import model_repository as mr
st = Session(url, token=access_token)
st

In [5]:
project_name = "Live_SAS_WBWorkshop"
repository_name = "DMRepository"

repository = mr.get_repository(repository_name)

project = mr.get_project(project_name)

if project == None:
    project = mr.create_project(project_name, repository)

In [6]:
import sasviya 
model = sasviya.load_model('artifacts/forest_model.pkl')

model_params = {
    "name": "RF_Model",
    "projectId": project.id,
    "type": "ASTORE"
}

astore = mr.post(
    "/models",
    files={"files": ("model_export.astore", model.export())},
    data=model_params,
)